In [1]:
!nvidia-smi

Fri Apr 28 10:04:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.84       Driver Version: 460.84       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:04:00.0 Off |                  N/A |
| 40%   72C    P2   153W / 250W |   4945MiB / 11178MiB |     63%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:06:00.0 Off |                  N/A |
| 43%   

In [2]:
import os
import torch
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, "0" to  "7" 
os.environ["CUDA_VISIBLE_DEVICES"]="4,5,6,7"
#torch.cuda.set_device(0)
n_gpu = torch.cuda.device_count()
print(n_gpu)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.get_device_name(0)


4


'GeForce GTX 1080 Ti'

In [3]:
import warnings 
warnings.filterwarnings('ignore')
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm


In [4]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation
#Set a seed
import random
import numpy as np
import torch
import datasets
import pandas as pd

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

from torch.utils.data import Dataset, DataLoader

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

[nltk_data] Downloading package punkt to /DATA/gupta92/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

set_seed(42)

In [6]:
#Loading dataset

In [7]:
pd.set_option('display.max_colwidth', None)

In [8]:
df = pd.read_csv('DATA/etoori_train.csv')
df.shape


(140000, 2)

In [9]:
#df = df.head(1000)

In [10]:
#df = pd.read_csv('DATA/Hindi_Artificial_train.csv')
#df.shape

In [11]:
#df = df.head(5000)

In [12]:
df.head(2)

,enc_input,dec_input
0,परन्तु वे दोनों उन बातों को ज़्यादा समय तक अप हृदय में गाँठ बनाकर नहीं रखते थे ।,परन्तु वे दोनों उन बातों को ज़्यादा समय तक अपने हृदय में गाँठ बनाकर नहीं रखते थे ।
1,देश में हिन्दी को विस्थापित कर का षड़यंत्र चल रहा है जो चिंता का विषय है ।,देश में हिन्दी को विस्थापित करने का षड़यंत्र चल रहा है जो चिंता का विषय है ।


In [13]:
df.rename(columns = {'enc_input':'input'}, inplace = True)
df.rename(columns = {'dec_input':'output'}, inplace = True)

In [14]:
#df = df.drop(['Unnamed: 0'], axis=1)

In [15]:
df.head(2)

,input,output
0,परन्तु वे दोनों उन बातों को ज़्यादा समय तक अप हृदय में गाँठ बनाकर नहीं रखते थे ।,परन्तु वे दोनों उन बातों को ज़्यादा समय तक अपने हृदय में गाँठ बनाकर नहीं रखते थे ।
1,देश में हिन्दी को विस्थापित कर का षड़यंत्र चल रहा है जो चिंता का विषय है ।,देश में हिन्दी को विस्थापित करने का षड़यंत्र चल रहा है जो चिंता का विषय है ।


In [16]:
from transformers import (
    T5ForConditionalGeneration, T5Tokenizer, 
    Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
  )

from torch.utils.data import Dataset, DataLoader

In [17]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
#tokenizer = AutoTokenizer.from_pretrained("google/mt5-base")
tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")

In [18]:
tokenizer

PreTrainedTokenizerFast(name_or_path='google/muril-base-cased', vocab_size=197285, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [19]:
tokenizer.model_max_length

512

In [20]:
#model = AutoModelForMaskedLM.from_pretrained("google/muril-base-cased")

In [21]:

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import MT5Model, AutoTokenizer,MT5ForConditionalGeneration

#model_name ='t5_gec_hindi_muRIL_best'
#model_name = "google/mt5-base"
#model_name = "mT5_Full_1"
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu' 

#model_name = "google/mt5-small"
model_name ="mT5_Etoori_2"
#tokenizer = T5Tokenizer.from_pretrained(model_name).to(torch_device)
model = MT5ForConditionalGeneration.from_pretrained(model_name).to(torch_device)

In [22]:
def translate(text, source_lang, target_lang):
    input_text = f"{source_lang} to {target_lang}: {text}"
    #input_ids = tokenizer.encode(input_text, return_tensors="pt").to(torch_device)
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(torch_device)
    outputs = model.generate(input_ids=input_ids, max_length=64, num_beams=4, early_stopping=True)
    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translated_text


In [23]:
text = "Hello, how are you?"
source_lang = "english"
target_lang = "french"
translation = translate(text, source_lang, target_lang)
print(translation)


##ंदक बी बी अर्धशतक बी बी अर्धशतक बी अर्धशतक बी अर्धशतक बी अर्धशतक बी अर्धशतक बी अर्धशतक बी अर्धशतक बी अर्धशतक बी अर्धशतक बी अर्धशतक बी अर्धशतक बी अर्धशतक बी अर्धशतक बी अर्धशतक बी अर्धशतक बी अर्धशतक बी अर्धशतक बी अर्धशतक बी अर्धशतक बी अर्धशतक बी अर्धशतक बी अर्धशतक बी अर्धशतक बी अर्धशतक बी अर्धशतक बी अर्धशतक बी अर्धशतक बी अर्धशतक बी


In [24]:
def calc_token_len(example):
    return len(tokenizer(example).input_ids)

In [25]:
#Train - Test Split
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.10, shuffle=True)

train_df.shape, test_df.shape

((126000, 2), (14000, 2))

In [26]:
test_df['input_token_len'] = test_df['input'].apply(calc_token_len)

In [27]:
test_df.head()

,input,output,input_token_len
40665,अँगरेज़ी सहभाषा के रूप में व्यवहत्त होनी चाहिए थी परंतु आज भी उसकी वर्चस्व सर्वत्र स्थापित है ।,अँगरेज़ी सहभाषा के रूप में व्यवहत्त होनी चाहिए थी परंतु आज भी उसका वर्चस्व सर्वत्र स्थापित है ।,28
48520,तथे गौशाला का,तथा गौशाला का,6
138403,बच्चे खेल रहे है किलकारियाँ गूंज रही है ब्रह्माण्ड में,बच्चे खेल रहे हैं किलकारियाँ गूंज रही हैं ब्रह्माण्ड में,14
130079,धरने पर किसानो ने कहा कि अब समय आ गया हैं कि गन्ने की खेती किसानो के लिये नुकसान का सौदा हो गयी हैं ।,धरने पर किसानो ने कहा कि अब समय आ गया है कि गन्ने की खेती किसानो के लिये नुकसान का सौदा हो गयी है ।,29
50146,सड़क मलबे का ढेर होने से दुपहिया व चार पहिया वाहन चालकों को परेशान होना पड़ रहा हैं ।,सड़क मलबे का ढेर होने से दुपहिया व चार पहिया वाहन चालकों को परेशान होना पड़ रहा है ।,23


In [28]:
test_df['input_token_len'].describe()

#train_df = df

count    14000.000000
mean        21.935000
std         12.422581
min          4.000000
25%         14.000000
50%         19.000000
75%         27.000000
max        308.000000
Name: input_token_len, dtype: float64

In [29]:
#We will use a token length of 64 since it will cover the vast majority of examples
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [30]:
train_df.input.str.len().max()

1823

In [31]:
#Load the Dataset
from torch.utils.data import Dataset, DataLoader
class GrammarDataset(Dataset):
    def __init__(self, dataset, tokenizer,print_text=False):         
        self.dataset = dataset
        self.pad_to_max_length = False
        self.tokenizer = tokenizer
        self.print_text = print_text
        #self.max_len = train_df.input.str.len().max()
        self.max_len = 64
  
    def __len__(self):
        return len(self.dataset)


    def tokenize_data(self, example):
        input_, target_ = example['input'], example['output']

        # tokenize inputs
        tokenized_inputs = tokenizer(input_, pad_to_max_length=self.pad_to_max_length, 
                                            max_length=self.max_len,
                                            return_attention_mask=True)
    
        tokenized_targets = tokenizer(target_, pad_to_max_length=self.pad_to_max_length, 
                                            max_length=self.max_len,
                                            return_attention_mask=True)

        inputs={"input_ids": tokenized_inputs['input_ids'],
            "attention_mask": tokenized_inputs['attention_mask'],
            "labels": tokenized_targets['input_ids']
        }
        
        return inputs

  
    def __getitem__(self, index):
        inputs = self.tokenize_data(self.dataset[index])
        
        if self.print_text:
            for k in inputs.keys():
                print(k, len(inputs[k]))

        return inputs

In [32]:
dataset = GrammarDataset(test_dataset, tokenizer, True)
print(dataset[10])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


input_ids 27
attention_mask 27
labels 27
{'input_ids': [104, 75159, 1154, 7899, 106524, 3768, 1169, 16015, 1250, 2316, 1419, 1123, 7132, 64219, 2115, 1125, 30361, 1438, 1228, 106524, 1117, 10171, 1194, 1254, 7544, 492, 105], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [104, 75159, 1154, 7899, 106524, 3768, 1169, 16015, 1250, 2316, 1243, 1123, 7132, 64219, 2115, 1125, 30361, 1438, 1228, 106524, 1117, 10171, 1194, 1254, 7544, 492, 105]}


In [33]:
#Define Evaluator
#!pip install rouge_score

In [34]:
from datasets import load_metric
rouge_metric = load_metric("rouge")

In [35]:
#Train Model
#data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding='longest', return_tensors='pt')
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding='longest', return_tensors='pt')


In [36]:
# defining training related arguments
batch_size = 1


args = Seq2SeqTrainingArguments(
    output_dir="models/hindi/mT5",
    #output_dir =None,
    evaluation_strategy="epoch",
    learning_rate=2e-3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.1,
    save_steps=300000,
    num_train_epochs=5,
    #dataloader_num_workers= 2,
    predict_with_generate=True,
    #bf16 = True
    #fp16=True,
    #logging_dir="logs/hindi/T5_muRIL_3"

)

In [37]:
# defining training related arguments
#batch_size = 8
'''
args = Seq2SeqTrainingArguments(output_dir="models/hindi/T5_muRIL_3",
                        evaluation_strategy="epoch",
                        per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,
                        learning_rate=2e-5,
                        num_train_epochs=1,
                        weight_decay=0.1,
                        save_total_limit=2,
                        predict_with_generate=True,
                        fp16 = True,
                        gradient_accumulation_steps = 6,
                        #eval_steps = 500,
                        #save_steps = 2000,
                        #dataloader_num_workers= 4,
                        load_best_model_at_end=True,
                        load_best_model_at_end=True,
                        metric_for_best_model="eval_loss",
                        greater_is_better=False,
                        early_stopping_patience=5,
                        early_stopping_threshold=0.01
                        logging_dir="logs/hindi/T5_muRIL_3")


args = Seq2SeqTrainingArguments(
    output_dir="models/hindi/mT5",
    #output_dir =None,
    evaluation_strategy="epoch",
    save_strategy ="epoch",
    learning_rate=2e-3,
    do_eval =True,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.1,
    save_steps=5000,
    eval_steps = 5000,
    num_train_epochs=1,
    #dataloader_num_workers= 2,
    predict_with_generate=True,
    #fp16=True,
    metric_for_best_model="eval_loss",
    #greater_is_better=False,
    load_best_model_at_end =True,
    #logging_dir="logs/hindi/T5_muRIL_3"

)
'''
print()

In [38]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
#os.environ["CUDA_LAUNCH_BLOCKING"] = "false"
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = 'true'

In [39]:
import torch
#torch.cuda.empty_cache()

In [40]:
# defining trainer using 🤗
trainer = Seq2SeqTrainer(model=model, 
                args=args, 
                train_dataset= GrammarDataset(train_dataset, tokenizer),
                eval_dataset=GrammarDataset(test_dataset, tokenizer),
                tokenizer=tokenizer,
                data_collator=data_collator)
                #compute_metrics=compute_metrics)

In [41]:
import os
os.environ["WANDB_DISABLED"] = "true"     #Disabling Wandb
#model.resize_token_embeddings(len(tokenizer))
trainer.train()

***** Running training *****
  Num examples = 126000
  Num Epochs = 5
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 157500
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss
1,0.802300,0.627577
2,0.672700,0.494649
3,0.483300,0.368540
4,0.298100,0.217544
5,0.120800,0.102051


***** Running Evaluation *****
  Num examples = 14000
  Batch size = 4
***** Running Evaluation *****
  Num examples = 14000
  Batch size = 4
***** Running Evaluation *****
  Num examples = 14000
  Batch size = 4
***** Running Evaluation *****
  Num examples = 14000
  Batch size = 4
***** Running Evaluation *****
  Num examples = 14000
  Batch size = 4


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=157500, training_loss=0.5695351831345331, metrics={'train_runtime': 93474.1085, 'train_samples_per_second': 6.74, 'train_steps_per_second': 1.685, 'total_flos': 2.176875283009536e+16, 'train_loss': 0.5695351831345331, 'epoch': 5.0})

In [ ]:
trainer.save_model('mT5_Etoori_3')

Saving model checkpoint to mT5_Etoori_3
Configuration saved in mT5_Etoori_3/config.json
Model weights saved in mT5_Etoori_3/pytorch_model.bin
tokenizer config file saved in mT5_Etoori_3/tokenizer_config.json
Special tokens file saved in mT5_Etoori_3/special_tokens_map.json


In [41]:
#import torch
#torch.cuda.empty_cache()

In [42]:
import os
os.environ["WANDB_DISABLED"] = "true"     #Disabling Wandb
#model.resize_token_embeddings(len(tokenizer))
trainer.train()

***** Running training *****
  Num examples = 126000
  Num Epochs = 5
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 157500
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss
1,0.691400,0.497808
2,0.604500,0.454815
3,0.433100,0.329243
4,0.276800,0.194706
5,0.112500,0.095339


***** Running Evaluation *****
  Num examples = 14000
  Batch size = 4
***** Running Evaluation *****
  Num examples = 14000
  Batch size = 4
***** Running Evaluation *****
  Num examples = 14000
  Batch size = 4
***** Running Evaluation *****
  Num examples = 14000
  Batch size = 4
***** Running Evaluation *****
  Num examples = 14000
  Batch size = 4


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=157500, training_loss=0.4741298280504015, metrics={'train_runtime': 94469.5906, 'train_samples_per_second': 6.669, 'train_steps_per_second': 1.667, 'total_flos': 2.176875283009536e+16, 'train_loss': 0.4741298280504015, 'epoch': 5.0})

In [43]:
trainer.save_model('mT5_Etoori_2')

Saving model checkpoint to mT5_Etoori_2
Configuration saved in mT5_Etoori_2/config.json
Model weights saved in mT5_Etoori_2/pytorch_model.bin
tokenizer config file saved in mT5_Etoori_2/tokenizer_config.json
Special tokens file saved in mT5_Etoori_2/special_tokens_map.json


In [ ]:
#Trained on 1k train data (Etoori)

In [47]:
import os
os.environ["WANDB_DISABLED"] = "true"     #Disabling Wandb
#model.resize_token_embeddings(len(tokenizer))
trainer.train()

***** Running training *****
  Num examples = 900
  Num Epochs = 5
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 3
  Gradient Accumulation steps = 1
  Total optimization steps = 1500


Epoch,Training Loss,Validation Loss
1,No log,0.693811
2,1.294400,0.514676
3,1.294400,0.486894
4,0.545500,0.451113
5,0.235400,0.418423


***** Running Evaluation *****
  Num examples = 100
  Batch size = 3
***** Running Evaluation *****
  Num examples = 100
  Batch size = 3
***** Running Evaluation *****
  Num examples = 100
  Batch size = 3
***** Running Evaluation *****
  Num examples = 100
  Batch size = 3
***** Running Evaluation *****
  Num examples = 100
  Batch size = 3


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1500, training_loss=0.6917728068033854, metrics={'train_runtime': 710.9826, 'train_samples_per_second': 6.329, 'train_steps_per_second': 2.11, 'total_flos': 142796496107520.0, 'train_loss': 0.6917728068033854, 'epoch': 5.0})

In [48]:
trainer.save_model('mT5_Etoori_1')

Saving model checkpoint to mT5_Etoori_1
Configuration saved in mT5_Etoori_1/config.json
Model weights saved in mT5_Etoori_1/pytorch_model.bin
tokenizer config file saved in mT5_Etoori_1/tokenizer_config.json
Special tokens file saved in mT5_Etoori_1/special_tokens_map.json


In [43]:
import os
os.environ["WANDB_DISABLED"] = "true"     #Disabling Wandb
#model.resize_token_embeddings(len(tokenizer))
trainer.train()

***** Running training *****
  Num examples = 126000
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 24
  Gradient Accumulation steps = 1
  Total optimization steps = 5250
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss
1,0.000000,nan


***** Running Evaluation *****
  Num examples = 14000
  Batch size = 24


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=5250, training_loss=0.0, metrics={'train_runtime': 5599.1057, 'train_samples_per_second': 22.504, 'train_steps_per_second': 0.938, 'total_flos': 1.8612324725170176e+16, 'train_loss': 0.0, 'epoch': 1.0})

In [ ]:
#torch_device = 'cuda' if torch.cuda.is_available() else 'cpu' 

In [52]:
print("Trained")

Trained


In [ ]:
!nvidia-smi

# Testing Trained Model

In [44]:
#Testing
import torch
from transformers import T5Tokenizer, MT5ForConditionalGeneration,AutoTokenizer
model_name = 'mT5_Etoori_2'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu' 

tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
#tokenizer = AutoTokenizer.from_pretrained("google/mt5-base")
print()

loading configuration file config.json from cache at /DATA/gupta92/.cache/huggingface/hub/models--google--muril-base-cased/snapshots/afd9f36c7923d54e97903922ff1b260d091d202f/config.json
Model config BertConfig {
  "_name_or_path": "google/muril-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.22.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 197285
}

loading file vocab.txt from cache at /DATA/gupta92/.cache/huggingface/hub/models--google--muril-base-cased/snapshots/afd9f36c7923d54e97903922ff1b260d091d202

In [45]:
trained_model = MT5ForConditionalGeneration.from_pretrained(model_name).to(torch_device)

loading configuration file mT5_Etoori_2/config.json
Model config MT5Config {
  "_name_or_path": "mT5_Etoori_1",
  "architectures": [
    "MT5ForConditionalGeneration"
  ],
  "d_ff": 1024,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "mt5",
  "num_decoder_layers": 8,
  "num_heads": 6,
  "num_layers": 8,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Tokenizer",
  "torch_dtype": "float32",
  "transformers_version": "4.22.2",
  "use_cache": true,
  "vocab_size": 250112
}

loading weights file mT5_Etoori_2/pytorch_model.bin
All model checkpoint weights were used when initializing MT5ForConditionalGeneration.

All the weights 

In [47]:
#trained_model1 = MT5ForConditionalGeneration.from_pretrained("google/Mt5-xl").to(torch_device)

In [ ]:
#trained_model

In [51]:
def gec_prediction(incorrect,correct):
    l = len(correct)
    predicted_s = correct_grammar(incorrect, num_return_sequences=4)[0]
    p = len(predicted_s)
    if(p>l):
        predicted_s = predicted_s[:l]
    elif (p<l-4):
        predicted_s = incorrect
    return predicted_s

In [47]:
param_size = 0
for param in trained_model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in trained_model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('model size: {:.3f}MB'.format(size_all_mb))

model size: 1145.083MB


In [48]:
def correct_grammar(input_text,num_return_sequences):
    #batch = tokenizer([input_text],truncation=True,padding='max_length',max_length=64, return_tensors="pt").to(torch_device)
    batch = tokenizer([input_text],truncation=True,padding='max_length',max_length=64, return_tensors="pt",return_token_type_ids=False).to(device)
    #translated= trained_model.generate(**batch,max_length=64,num_beams=4, num_return_sequences=num_return_sequences, temperature=1.5)
    translated= trained_model.generate(**batch,max_length=64,num_beams=4, num_return_sequences=num_return_sequences, temperature=1.5)
    #print("here")
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_text

In [58]:
#print(T5ForConditionalGeneration.__version__)

In [52]:
text = test_df['input'].iat[100]
correct = test_df['output'].iat[100]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)

pred = gec_prediction(text,correct)
print("Predicted Sentence:",pred)

Incorrect Sentence: प्रशासन देरी से पहुंचे और बिना सूचना के गैरहाजिर डॉक्टरों को नोटिस जारी किए हैं ।
Correct Seentence: प्रशासन ने देरी से पहुंचे और बिना सूचना के गैरहाजिर डॉक्टरों को नोटिस जारी किए हैं । 
Predicted Sentence: प्रशासन ने देरी से पहुंचे और बिना सूचना के गैरहाजिर डॉक्टरों को नोटिस जारी किए हैं । 


In [53]:
text = test_df['input'].iat[1000]
correct = test_df['output'].iat[1000]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)

pred = gec_prediction(text,correct)
print("Predicted Sentence:",pred)

Incorrect Sentence: इससे केसी काम्प्लेक्स में करवाए गए विभिन्न निर्माण अवैध की श्रेणी में आते है ।
Correct Seentence: इससे केसी काम्प्लेक्स में करवाए गए विभिन्न निर्माण अवैध की श्रेणी में आते हैं । 
Predicted Sentence: इससे केसी काम्प्लेक्स में करवाए गए विभिन्न निर्माण अवैध की श्रेणी में आते हैं । 


# Performance Analysis
1. BLEU Score
2. GLEU Score

In [ ]:
test_df.shape

In [ ]:
test_df.head(5)

In [ ]:
test_df

In [54]:
from nltk.translate.gleu_score import sentence_gleu
GLEU_val_emb = []
test_data = test_df.head(100)
print(test_data.shape)
itr = 0
for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        itr+=1
        text = str(i.input)
        #print(text)
        #pred = predict(str(i.enc_input),model)[0].split()
        pred = correct_grammar(text, num_return_sequences=1)[0]
        l = len(text)
        p = len(pred)
        if(p>l):
            pred = pred[:l]
            
        act = [str(i.output).split()]
        pred_s = str(pred).split()
        b = sentence_gleu(act,pred_s)
        GLEU_val_emb.append(b)
        if(itr%100 ==0):
            print("GELU Score for ",itr," data point",np.mean(GLEU_val_emb))
    except:
        print("Error") 
        continue
print("GELU Score = ",np.mean(GLEU_val_emb))

(100, 3)


100it [01:43,  1.03s/it]

GELU Score for  100  data point 0.8816151651320783
GELU Score =  0.8816151651320783


# Evaluation on Etoori's Test Dataset

In [56]:
import pandas as pd
df_test = pd.read_csv('DATA/etoori_test.csv')
df_test.shape

(30000, 2)

In [57]:
df_test.head(5)

,enc_input,dec_input
0,इसके अलावा माइकल शूमाकर द्वारा चलाई गई एक फरारी कार भी को मिलियन डॉलर क्लब में माना जा रहा है ।,इसके अलावा माइकल शूमाकर द्वारा चलाई गई एक फरारी कार को भी मिलियन डॉलर क्लब में माना जा रहा है ।
1,यह मन को काबू में करने वाली मुद्रा हैं इसीलिए इसे चित्त हस्त मुद्रा योग कहते हैंं ।,यह मन को काबू में करने वाली मुद्रा है इसीलिए इसे चित्त हस्त मुद्रा योग कहते हैं ।
2,आप पुस्तक पढ़ने में तल्लीन हैं और बच्चा उसनेाँ पर शोर मचा रहा है ।,आप पुस्तक पढ़ने में तल्लीन हैं और बच्चा वहाँ पर शोर मचा रहा है ।
3,परिवार के मुताबिक धमाकों में हिस्सा लेने वाले कोई दूसरे लोग थे और ये गलत पहचान का मामला हैं ।,परिवार के मुताबिक धमाकों में हिस्सा लेने वाले कोई दूसरे लोग थे और ये गलत पहचान का मामला है ।
4,उनकी वो वाली बात भी अनिश्चित रहती हैं ।,उनकी वो वाली बात भी अनिश्चित रहती है ।


In [58]:
text = df_test['enc_input'].iat[0]
correct = df_test['dec_input'].iat[0]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]

print("Predicted Sentence:",predicted_s)

Incorrect Sentence: इसके अलावा माइकल शूमाकर द्वारा चलाई गई एक फरारी कार भी को मिलियन डॉलर क्लब में माना जा रहा है ।
Correct Seentence: इसके अलावा माइकल शूमाकर द्वारा चलाई गई एक फरारी कार को भी मिलियन डॉलर क्लब में माना जा रहा है । 
Predicted Sentence: इसके अलावा माइकल शूमाकर द्वारा चलाई गई एक फरारी कार को भी मिलियन डॉलर क्लब में माना जा रहा है । 


In [59]:
text = df_test['enc_input'].iat[50]
correct = df_test['dec_input'].iat[50]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]

print("Predicted Sentence:",predicted_s)

Incorrect Sentence: घर में घुस के बाद भी सुकून नहीं ।
Correct Seentence: घर में घुसने के बाद भी सुकून नहीं । 
Predicted Sentence: घर में घुसने के बाद भी सुकून नहीं । 


In [60]:
%%time
predicted_s = correct_grammar(text, num_return_sequences=1)[0]

CPU times: user 999 ms, sys: 52.1 ms, total: 1.05 s
Wall time: 1.05 s


In [ ]:
import nltk.translate.bleu_score as bleu
BLEU = []
index = []
test_data = df_test.head(10000)
np.random.seed(1)
for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        text = str(i.enc_input)
        #pred = predict(str(i.enc_input),model)[0].split()
        pred = correct_grammar(text, num_return_sequences=1)[0]
        act = [str(i.dec_input).split()]
        pred_s = str(pred).split()
        #print(act)
        #print(pred_s)
        b = bleu.sentence_bleu(act,pred_s)
        BLEU.append(b)
    except:
        index.append(ind)
        continue
print("BELU = ", np.mean(BLEU))

In [61]:
from nltk.translate.gleu_score import sentence_gleu
GLEU_val_emb = []
test_data = df_test.head(1000)
print(test_data.shape)
for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        text = str(i.enc_input)
        #pred = predict(str(i.enc_input),model)[0].split()
        pred = correct_grammar(text, num_return_sequences=1)[0]
        l = len(text)
        p = len(pred)
        if(p>l):
            pred = pred[:l]
            
        act = [str(i.dec_input).split()]
        pred_s = str(pred).split()
    
        #print(act)
        #print(pred_s)
        b = sentence_gleu(act,pred_s)
        #print(b)
        GLEU_val_emb.append(b)
    except:
        continue
print("GELU Score = ",np.mean(GLEU_val_emb))

(1000, 2)


1000it [17:39,  1.06s/it]

GELU Score =  0.9016537992385885


In [62]:
from nltk.translate.gleu_score import sentence_gleu
GLEU_val_emb = []
test_data = df_test.head(10000)
print(test_data.shape)
for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        text = str(i.enc_input)
        #pred = predict(str(i.enc_input),model)[0].split()
        pred = correct_grammar(text, num_return_sequences=1)[0]
        l = len(text)
        p = len(pred)
        if(p>l):
            pred = pred[:l]
            
        act = [str(i.dec_input).split()]
        pred_s = str(pred).split()
        b = sentence_gleu(act,pred_s)
        GLEU_val_emb.append(b)
        if(ind%1000 ==0):
            print("GELU Score for ",ind+1," data point",np.mean(GLEU_val_emb))
    except:
        continue
print("GELU Score = ",np.mean(GLEU_val_emb))

(10000, 2)


1it [00:01,  1.05s/it]

GELU Score for  1  data point 1.0


1001it [17:34,  1.29s/it]

GELU Score for  1001  data point 0.9017520471913971


...

GELU Score for  8001  data point 0.9031189247194619


9001it [2:34:34,  1.01s/it]

GELU Score for  9001  data point 0.9025801389217523


10000it [2:51:36,  1.03s/it]

GELU Score =  0.9023208220868466


In [63]:
print("GELU Score = ",np.mean(GLEU_val_emb))

GELU Score =  0.9023208220868466


# Testing HiWikEd test dataset

In [ ]:
import pandas as pd
df_test1 = pd.read_csv('DATA/HiWikEd.csv')
df_test1.shape
df_test1 = df_test1.drop(['Unnamed: 0'], axis=1)

In [ ]:
df_test1.head(5)

In [ ]:
text = df_test1['enc_input'].iat[45]
correct = df_test1['dec_input'].iat[45]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]

print("Predicted Sentence:",predicted_s)

In [ ]:
text = df_test1['enc_input'].iat[40]
correct = df_test1['dec_input'].iat[40]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]

print("Predicted Sentence:",predicted_s)

In [ ]:
import warnings 
warnings.filterwarnings('ignore')
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm
import numpy as np
from nltk.translate.gleu_score import sentence_gleu


GLEU_val_emb = []
test_data = df_test1.head(100)
print(test_data.shape)

for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        text = str(i.enc_input)
        #pred = predict(str(i.enc_input),model)[0].split()
        pred = correct_grammar(text, num_return_sequences=1)[0]
        l = len(text)
        p = len(pred)
        if(p>l):
            pred = pred[:l]
            
        act = [str(i.dec_input).split()]
        pred_s = str(pred).split()
        b = sentence_gleu(act,pred_s)
        GLEU_val_emb.append(b)
        if(ind%100 ==0):
            print("GELU Score for ",ind+1," data point",np.mean(GLEU_val_emb))
    except:
        continue
print("GELU Score = ",np.mean(GLEU_val_emb))

In [ ]:
import pandas as pd
df_test_pron_infl = pd.read_csv('DATA/Wikiedits_PRON_INFL.csv')
print("PRON_INFL:", df_test_pron_infl.shape)

df_test_adj_infl = pd.read_csv('DATA/Wikiedits_ADJ_INFL.csv')
print("ADJ_INFL:", df_test_adj_infl.shape)

df_test_adp_infl = pd.read_csv('DATA/Wikiedits_ADP_INFL.csv')
print("ADP_INFL:", df_test_adp_infl.shape)

df_test_verb_infl = pd.read_csv('DATA/Wikiedits_VERB_INFL.csv')
print("VERB_INFL:", df_test_verb_infl.shape)

df_test_noun_infl = pd.read_csv('DATA/Wikiedits_NOUN_INFL.csv')
print("NOUN_INFL:", df_test_noun_infl.shape)

In [ ]:
df_test_pron_infl.head(2)

In [ ]:
df_test_adj_infl.head(2)

In [ ]:
import warnings 
warnings.filterwarnings('ignore')
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm
import numpy as np
from nltk.translate.gleu_score import sentence_gleu


GLEU_val_emb = []
test_data = df_test_adj_infl
print(test_data.shape)

for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        text = str(i.enc_input)
        #pred = predict(str(i.enc_input),model)[0].split()
        pred = correct_grammar(text, num_return_sequences=1)[0]
        l = len(text)
        p = len(pred)
        if(p>l):
            pred = pred[:l]
            
        act = [str(i.dec_input).split()]
        pred_s = str(pred).split()
        b = sentence_gleu(act,pred_s)
        GLEU_val_emb.append(b)
        if(ind%50 ==0):
            print("GELU Score for ",ind+1," data point",np.mean(GLEU_val_emb))
    except:
        continue
print("GELU Score = ",np.mean(GLEU_val_emb))

In [ ]:
print("GELU Score for ADJ_INFL Error= ",np.mean(GLEU_val_emb))

In [ ]:
import warnings 
warnings.filterwarnings('ignore')
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm
import numpy as np
from nltk.translate.gleu_score import sentence_gleu


GLEU_val_emb = []
test_data = df_test_pron_infl
print(test_data.shape)

for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        text = str(i.enc_input)
        #pred = predict(str(i.enc_input),model)[0].split()
        pred = correct_grammar(text, num_return_sequences=1)[0]
        l = len(text)
        p = len(pred)
        if(p>l):
            pred = pred[:l]
            
        act = [str(i.dec_input).split()]
        pred_s = str(pred).split()
        b = sentence_gleu(act,pred_s)
        GLEU_val_emb.append(b)
        if(ind%100 ==0):
            print("GELU Score for ",ind+1," data point",np.mean(GLEU_val_emb))
    except:
        continue
print("GELU Score = ",np.mean(GLEU_val_emb))

In [ ]:
print("GELU Score for PRON_INFL Error= ",np.mean(GLEU_val_emb))

In [ ]:
import warnings 
warnings.filterwarnings('ignore')
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm
import numpy as np
from nltk.translate.gleu_score import sentence_gleu


GLEU_val_emb = []
test_data = df_test_adp_infl.head(100)
print(test_data.shape)

for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        text = str(i.enc_input)
        #pred = predict(str(i.enc_input),model)[0].split()
        pred = correct_grammar(text, num_return_sequences=1)[0]
        l = len(text)
        p = len(pred)
        if(p>l):
            pred = pred[:l]
            
        act = [str(i.dec_input).split()]
        pred_s = str(pred).split()
        b = sentence_gleu(act,pred_s)
        GLEU_val_emb.append(b)
        if(ind%200 ==0):
            print("GELU Score for ",ind+1," data point",np.mean(GLEU_val_emb))
    except:
        continue
print("GELU Score = ",np.mean(GLEU_val_emb))

In [ ]:
print("GELU Score for ADP_INFL Error= ",np.mean(GLEU_val_emb))

In [ ]:
import warnings 
warnings.filterwarnings('ignore')
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm
import numpy as np
from nltk.translate.gleu_score import sentence_gleu


GLEU_val_emb = []
test_data = df_test_adp_infl
print(test_data.shape)

for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        text = str(i.enc_input)
        #pred = predict(str(i.enc_input),model)[0].split()
        pred = correct_grammar(text, num_return_sequences=1)[0]
        l = len(text)
        p = len(pred)
        if(p>l):
            pred = pred[:l]
            
        act = [str(i.dec_input).split()]
        pred_s = str(pred).split()
        b = sentence_gleu(act,pred_s)
        GLEU_val_emb.append(b)
        if(ind%200 ==0):
            print("GELU Score for ",ind+1," data point",np.mean(GLEU_val_emb))
    except:
        continue
print("GELU Score = ",np.mean(GLEU_val_emb))

In [ ]:
print("GELU Score for ADP_INFL Error= ",np.mean(GLEU_val_emb))

In [ ]:
import warnings 
warnings.filterwarnings('ignore')
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm
import numpy as np
from nltk.translate.gleu_score import sentence_gleu


GLEU_val_emb = []
test_data = df_test_verb_infl
print(test_data.shape)

for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        text = str(i.enc_input)
        #pred = predict(str(i.enc_input),model)[0].split()
        pred = correct_grammar(text, num_return_sequences=1)[0]
        l = len(text)
        p = len(pred)
        if(p>l):
            pred = pred[:l]
            
        act = [str(i.dec_input).split()]
        pred_s = str(pred).split()
        b = sentence_gleu(act,pred_s)
        GLEU_val_emb.append(b)
        if(ind%1000 ==0):
            print("GELU Score for ",ind+1," data point",np.mean(GLEU_val_emb))
    except:
        continue
print("GELU Score = ",np.mean(GLEU_val_emb))

In [ ]:
print("GELU Score for VERB_INFL Error= ",np.mean(GLEU_val_emb))

In [ ]:
import warnings 
warnings.filterwarnings('ignore')
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm
import numpy as np
from nltk.translate.gleu_score import sentence_gleu


GLEU_val_emb = []
test_data = df_test_noun_infl
print(test_data.shape)

for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        text = str(i.enc_input)
        #pred = predict(str(i.enc_input),model)[0].split()
        pred = correct_grammar(text, num_return_sequences=1)[0]
        l = len(text)
        p = len(pred)
        if(p>l):
            pred = pred[:l]
            
        act = [str(i.dec_input).split()]
        pred_s = str(pred).split()
        b = sentence_gleu(act,pred_s)
        GLEU_val_emb.append(b)
        if(ind%50 ==0):
            print("GELU Score for ",ind+1," data point",np.mean(GLEU_val_emb))
    except:
        continue
print("GELU Score = ",np.mean(GLEU_val_emb))

In [ ]:
print("GELU Score for NOUN_INFL Error= ",np.mean(GLEU_val_emb))

In [ ]:
df_test1.shape

In [ ]:
import warnings 
warnings.filterwarnings('ignore')
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm
import numpy as np
from nltk.translate.gleu_score import sentence_gleu


GLEU_val_emb = []
test_data = df_test1.head(100)
print(df_test1.shape)

for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        text = str(i.enc_input)
        #pred = predict(str(i.enc_input),model)[0].split()
        pred = correct_grammar(text, num_return_sequences=1)[0]
        l = len(text)
        p = len(pred)
        if(p>l):
            pred = pred[:l]
            
        act = [str(i.dec_input).split()]
        pred_s = str(pred).split()
        b = sentence_gleu(act,pred_s)
        GLEU_val_emb.append(b)
        if(ind%20 ==0):
            print("GELU Score for ",ind+1," data point",np.mean(GLEU_val_emb))
    except:
        continue
print("GELU Score = ",np.mean(GLEU_val_emb))

In [ ]:
import warnings 
warnings.filterwarnings('ignore')
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm
import numpy as np
from nltk.translate.gleu_score import sentence_gleu


GLEU_val_emb = []
test_data = df_test1
print(df_test1.shape)

for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        text = str(i.enc_input)
        #pred = predict(str(i.enc_input),model)[0].split()
        pred = correct_grammar(text, num_return_sequences=1)[0]
        l = len(text)
        p = len(pred)
        if(p>l):
            pred = pred[:l]
            
        act = [str(i.dec_input).split()]
        pred_s = str(pred).split()
        b = sentence_gleu(act,pred_s)
        GLEU_val_emb.append(b)
        if(ind%1000 ==0):
            print("GELU Score for ",ind+1," data point",np.mean(GLEU_val_emb))
    except:
        continue
print("GELU Score = ",np.mean(GLEU_val_emb))

In [ ]:
# F0.5 Score

In [ ]:
from sklearn.metrics import fbeta_score

# Define true translations and predicted translations for a set of source texts
true_translations = ["इसे गौरी शंकर मंदिर भी कहते हैं."]
predicted_translations = ["इसे गौरी शंकर मंदिर भी कहते हैं."]

# Calculate the F0.5 score
f_score = fbeta_score(true_translations, predicted_translations, beta=0.5, average='weighted')

print("F0.5 Score: {:.2f}".format(f_score))